# Project 3 - Part 3

## Specifications - Database
- Your stakeholder wants you to take the data you have been cleaning and collecting in Parts 1 & 2 of the project, and wants you to create a MySQL database for them.

- Specifically, they want the data from the following files included in your database:
    - Title Basics:
        - Movie ID (tconst)
        - Primary Title
        - Start Year
        - Runtime (in Minutes)
        - Genres
    - Title Ratings
        - Movie ID (tconst)
        - Average Movie Rating
        - Number Of Votes
    - The TMDB API Results (multiple files)
        - Movie ID
        - Revenue
        - Budget
        - Certification (MPAA Rating)

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Import/Install pymysql
import pymysql
pymysql.install_as_MySQLdb()

# sqlalchemy libraries
from sqlalchemy import create_engine
from sqlalchemy.types import *
from sqlalchemy_utils import create_database, database_exists

In [2]:
# load basics file
df = pd.read_csv('Data/title_basics.csv.gz')

Required Transformation Steps for Title Basics:
**Normalize Genre:**

Convert the single string of genres from title basics into 2 new tables.
    1. title_genres: with the columns:

    - tconst
    - genre_id
2. genres:

    - genre_id
    - genre_name

In [3]:
df.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama


## 1. Getting List of Unique Genres

In [4]:
# Create new column by seperating 'genres'
# split by the comma (,) in 'genres'
df['genres_split'] = df['genres'].str.split(',')
df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama,[Drama]
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,[Drama]
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama,[Drama]
...,...,...,...,...,...,...,...,...,...,...
84963,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama,[Drama]
84964,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
84965,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama,[Drama]
84966,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"


In [5]:
exploded_genres = df.explode('genres_split')
exploded_genres

# genres_split no longer has [] & is seperated by genres

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Romance
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,Drama
...,...,...,...,...,...,...,...,...,...,...
84966,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller",Action
84966,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller",Adventure
84966,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller",Thriller
84967,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History",Drama


In [6]:
unique_genres = sorted(exploded_genres['genres_split'].unique())
unique_genres

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

## 2. Create a new title_genres table

In [7]:
## Save just tconst & genres_split as new df
title_genres = exploded_genres[['tconst', 'genres_split']].copy()
title_genres.head(3)

,tconst,genres_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance


## 3. Create a genre mapper dictionary to replace string genres with integers

- Reminder:  we will use the dict and zip functions to make a dictionary with the genre strings as the keys and the integers genre_ids as the values.

In [8]:
## Making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Short': 20,
 'Sport': 21,
 'Talk-Show': 22,
 'Thriller': 23,
 'War': 24,
 'Western': 25}

In [9]:
## Make a dict with list of unique genres as the key & the new integer id as values
genre_id_map = dict(zip(unique_genres, range(len(unique_genres))))
genre_id_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Short': 20,
 'Sport': 21,
 'Talk-Show': 22,
 'Thriller': 23,
 'War': 24,
 'Western': 25}

## 4. Replace the string genres in title_genres with the new integer ids.
- Create a new genre_id column using the genre_map dict with either .map or .replace
- Next, drop the "genres_split" column
- Your title_genres table is now ready for your MySQL database!

In [10]:
## make new integer genre_id and drop string genres
df['genre_id'] = df['genres_split'].replace(genre_map)
df = df.drop(columns='genres_split')

#.map did not work for me, so I used .replace

## 5. Convert the genre map dictionary into a dataframe.


In [12]:
## Manually make dataframe with named cols from the .keyd and .values
genre_lookup = pd.DataFrame({'Genre_Name': genre_id_map.keys(),
                            'Genre_ID': genre_id_map.values()})
genre_lookup.head()

,Genre_Name,Genre_ID
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


---

# II) Saving the MySQL tables with tconst as the primary key.

In [13]:
df.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult             int64
startYear         float64
endYear           float64
runtimeMinutes      int64
genres             object
genre_id           object
dtype: object